In [ ]:
import pickle

data_path = '/mnt/userdata/liangsirui/MyProject/dmodel/save_hidden_states/data0/reft/layer_31.pkl'
with open(data_path, 'rb') as f:
    loaded_data = pickle.load(f)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="Results/Llama3/1000_attn_o_2e-05/log/events.out.tfevents.1744179824.pai-a100-156.11502.0")

In [2]:
import json
import os

def read_json_file(file_path):
    try:
        # 以只读模式打开文件，使用 UTF-8 编码
        with open(file_path, 'r', encoding='utf-8') as file:
            # 加载 JSON 数据
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"文件 {file_path} 未找到。")
    except json.JSONDecodeError:
        print(f"文件 {file_path} 不是有效的 JSON 格式。")

data = "/ReFT/dataset/gsm8k/test.json"
# data = "/ReFT/dataset/math_10k/train.json"

data_path = "/mnt/userdata/liangsirui/MyProject"
if not os.path.exists(data_path):
    data_path = "/mnt/userdata/MyProject"

file_path = data_path + data

question_list = read_json_file(file_path) 

In [18]:
import torch

def load_pth_file(file_path):
    """
    读取 PyTorch .pth 文件并返回存储的内容
    """
    data = torch.load(file_path, map_location=torch.device('cuda'))
    return data

data1 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1/ffn_up/delta_vector.pth')
data2 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1e-1/ffn_up/delta_vector.pth')
data3 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1e-2/ffn_up/delta_vector.pth')
data4 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1e-3/ffn_up/delta_vector.pth')
data5 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1e-4/ffn_up/delta_vector.pth')
data6 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1e-5/ffn_up/delta_vector.pth')
data7 = load_pth_file('Results/Test/Llama3/5000_math10k_all_64_2e-4_1e-6/ffn_up/delta_vector.pth')

data1_norm, data2_norm, data3_norm, data4_norm, data5_norm, data6_norm, data7_norm = [],[],[],[],[],[],[]
for layer in range(32):
    l2_norm1 = torch.norm(data1[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)
    l2_norm2 = torch.norm(data2[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)
    l2_norm3 = torch.norm(data3[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)
    l2_norm4 = torch.norm(data4[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)
    l2_norm5 = torch.norm(data5[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)
    l2_norm6 = torch.norm(data6[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)
    l2_norm7 = torch.norm(data7[f'model.layers.{layer}.mlp.up_proj.delta_vector.activation_bias'][0], p=2)

    data1_norm.append(round(l2_norm1.item(),3))
    data2_norm.append(round(l2_norm2.item(),3))
    data3_norm.append(round(l2_norm3.item(),3))
    data4_norm.append(round(l2_norm4.item(),3))
    data5_norm.append(round(l2_norm5.item(),3))
    data6_norm.append(round(l2_norm6.item(),3))
    data7_norm.append(round(l2_norm7.item(),3))

print("1:", sum(data1_norm)/32)
print("1e-1:", sum(data2_norm)/32)
print("1e-2:", sum(data3_norm)/32)
print("1e-3:", sum(data4_norm)/32)
print("1e-4:", sum(data5_norm)/32)
print("1e-5:", sum(data6_norm)/32)
print("1e-6:", sum(data7_norm)/32)



1: 0.7820312500000001
1e-1: 0.5863750000000001
1e-2: 0.54946875
1e-3: 0.55478125
1e-4: 0.5402499999999999
1e-5: 0.48837499999999995
1e-6: 0.3393749999999999


In [4]:
def extract_response(text: str) -> str:
    start_marker = "### Response:"
    start_idx = text.find(start_marker)
    
    if start_idx == -1:
        return ""
    
    response_text = text[start_idx + len(start_marker):].strip()
    
    end_marker = "assistant"
    end_idx = response_text.find(end_marker)

    if end_idx != -1:
        second_end_idx = response_text.find(end_marker, end_idx + len(end_marker))
        if second_end_idx != -1:
            response_text = response_text[:second_end_idx].strip()
    
    response_text.replace('assistant', '')
    response_text.strip()
    
    return response_text

In [16]:
data_path0 = "Results/Llama3/9000_attn_o_2e-5/template1/gsm8k_eval.json"
data_path1 = "Results/Llama3/9000_lora/template1/gsm8k_eval.json"

red = read_json_file(data_path0)
lora = read_json_file(data_path1)

count = 0
for data_index in red.keys():
    if "attn_o" not in red[data_index].keys():
        break
    if red[data_index]['eval'] == True and lora[data_index]['eval']  == False:
        print('--------------------------------------------------------------')
        print("Question")
        print(red[data_index]["Question"])
        print("Answer")
        print(red[data_index]["Answer"])

        print('【red】')
        print(extract_response(red[data_index]["attn_o"]))
        print('【lora】')
        print(extract_response(lora[data_index]["lora"]))

        count+=1

print(count)
        

--------------------------------------------------------------
Question
Naruto can lift a mountain ten times higher than Kagiyami can.  But Kagiyami can lift a mountain 4 times higher than Saskay can.  And Saskay can lift a mountain 12 times higher than Pompei can.  If Pompei can lift a mountain 1 inch, how high can Naruto lift a mountain, in feet?
Answer
40.0
【red】
assistant

Let's start by finding out how many inches Saskay can lift:
Pompei can lift a mountain = 1 inch
Saskay can lift a mtn 12 times higher = 12 x 1 = 12 inches

Next, find out how many inches Kagiyami can lifte:
Kagiyami can lift 4 times higher = 4 x 12 = 48 inches

Finally, find out how much Naruto can lift:
Naruto can lift 10 times higher = 10 x 48 = 480 inches

To convert this to feet, divide by 12 (since there are 12 inches in a foot):
480 / 12 = 40 feet

So Naruto can lift a mountain that is 40 feet high.
Final Answer: The final answer is 40. I hope it is correct.
【lora】
assistant

Let's start by finding out how 